In [ ]:
!mkdir datasets
!ls D:\DevProjects\CSEFinal\ProjectFiles\asl_alphabet_train\asl_alphabet_train

In [ ]:
import splitfolders
train_data = "asl_alphabet_train/asl_alphabet_train"

splitfolders.ratio(train_data, output="datasets/asl_alphabet", 
                   seed=1337, ratio=(.8,.1,.1), group_prefix=None, move=False)

In [1]:
train_data = "asl_alphabet_train/asl_alphabet_train"
train_dir = 'datasets/asl_alphabet/train'
val_dir = 'datasets/asl_alphabet/val'
test_dir  = 'datasets/asl_alphabet/test'

In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms

batch_size = 32
target_size = (32, 32)

# Define data transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Grayscale conversion
    transforms.Resize(target_size),  # Resize images
    transforms.ToTensor(),  # Convert to PyTorch tensors
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Normalize pixel values
])

# Create datasets
train_dataset = torchvision.datasets.ImageFolder(train_dir, transform=transform)
val_dataset = torchvision.datasets.ImageFolder(val_dir, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# Create data loaders with shuffling for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Create data loader for testing (no shuffling needed)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [3]:
labels = list(train_dataset.class_to_idx.keys())
print(labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


## Show dataset

In [6]:
from torch.utils.data import DataLoader
import torch

# Assuming you have your data loader defined:
data_loader = train_loader  # Your data loader

# Create an empty tensor to store images
num_images_to_sample = 8  # Adjust as needed
images_tensor = torch.empty((num_images_to_sample, *images_batch[0].shape))  # Match image shape

# Sample images from the data loader
sampled_images = 0
for images_batch, _ in data_loader:
  # Concatenate the batch with the existing tensor (up to the desired number)
  images_tensor[:min(num_images_to_sample - sampled_images, len(images_batch))] = images_batch[:min(num_images_to_sample - sampled_images, len(images_batch))]
  sampled_images += len(images_batch)
  if sampled_images >= num_images_to_sample:
    break  # Break after sampling enough images

# Create and display the grid (same as before)
grid = make_grid(images_tensor, nrow=4)
plt.imshow(grid.permute(1, 2, 0))
plt.axis('off')
plt.show()


NameError: name 'images_batch' is not defined

In [ ]:
sample_img(labels[:10])

## Build Model

In [4]:
import torch
from torch import nn

In [5]:
num_classes = len(labels)
in_shape = (32,32,1)

In [10]:

# Define input shape (assuming channels, height, width)
in_channels, in_height, in_width = in_shape

# Create the model class
class ConvolutionalNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    
    # Convolutional layers
    self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)  # Same padding
    self.bn1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(64)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
    self.bn3 = nn.BatchNorm2d(128)

    # Fully-connected layers
    self.fc1 = nn.Linear(in_height * in_width * 128, 128)  # Flatten before FC
    self.fc2 = nn.Linear(128, num_classes)

  def forward(self, X):
    X = F.relu(self.conv1(x))
    x = F.pool1(x)
    x = F.bn1(x)
    x = F.relu(self.conv2(x))
    x = F.pool2(x)
    x = F.bn2(x)
    x = F.relu(self.conv3(x))
    x = F.pool3(x)
    x = F.bn3(x)
    x = x.view(-1, in_height * in_width * 128)  # Flatten
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

# Create an instance of the model
torch.manual_seed(42)
model = ConvolutionalNetwork()

# Print model summary (similar to TensorFlow summary)
print(model)


ConvolutionalNetwork(
  (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=29, bias=True)
)


In [14]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(6*6*64, 600)
        self.dropout = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(600, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, X):
        X = F.relu(self.bn1(self.conv1(X)))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.bn2(self.conv2(X)))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 6*6*64)
        X = F.relu(self.fc1(X))
        x = self.dropout(X)
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=1)

In [11]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Using GPU for training!")
else:
  device = torch.device("cpu")
  print("Using CPU for training (GPU not available).")

Using GPU for training!


In [12]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')
    
count_parameters(model)

  9216
    32
    32
    32
 18432
    64
    64
    64
 73728
   128
   128
   128
524288
   128
  3712
    29
______
630205


In [13]:
from torch.optim import Adam

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer with a learning rate of 0.001
optimizer = Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50  # Set the number of epochs to 50

for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):  # Assuming train_loader is your data loader
    # Forward pass
    outputs = model(images)

    # Calculate loss
    loss = criterion(outputs, labels)

    # Backward pass and update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print training progress (optional)
    if (i+1) % print_every == 0:  # Print every 'print_every' batches
      print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


NameError: name 'F' is not defined